In [1]:
import os
import numpy as np
from cycler import cycler

import matplotlib as mpl
import matplotlib.pyplot as plt

from astropy.io import fits
from astropy.table import Table, join, hstack
import astropy.units as u

import scipy.stats as stats

plt.rcParams.update({
    'figure.figsize'   : (12, 10),
    'font.family'      :'serif',
    'font.size'        : 24,
    'font.serif'       :'Georgia',
    'axes.labelsize'   :'large',
    'mathtext.fontset' :'stix',
    'axes.linewidth'   : 5,
    'xtick.direction'  :'in',
    'ytick.direction'  :'in',
    'xtick.major.size' : 7,
    'ytick.major.size' : 7,
    'xtick.major.width': 3,
    'ytick.major.width': 3,
    'xtick.minor.size' : 5,
    'ytick.minor.size' : 5,
    'xtick.minor.width': 2,
    'ytick.minor.width': 2,
    'lines.linewidth'  : 3,
    'axes.prop_cycle'  : cycler('color', ['red', 'orange', 'teal', 'skyblue']),
})

def errJy_to_errABmag(flux_Jy, fluxerr_Jy):
    return -2.5 / (flux_Jy * np.log(10)) * fluxerr_Jy

## SOM-ready panchromatic catalog

In [2]:
with fits.open('/data2/lsajkov/mpdg/data/COSMOS/COSMOS2020_CLASSIC_R1_v2.2_p3.fits') as cat:
    COSMOSClassicCat = Table(cat[1].data)

In [23]:
#construct SOM catalog columns for COSMOS2020 Classic
cat = COSMOSClassicCat

IDs = cat['ID']

photometric_bands = ['CFHT_u', 'HSC_g', 'HSC_r', 'HSC_i', 'HSC_z', 'UVISTA_Y', 'UVISTA_J', 'UVISTA_H', 'UVISTA_Ks']
ra = cat['ALPHA_J2000']
dec = cat['DELTA_J2000']

magnitudes = Table()
aper_magnitudes = Table()
magnitude_errs = Table()
SNRs = Table()

for band in photometric_bands:
    magnitudes.add_column((cat[f'{band}_MAG_AUTO']),
                          name = f'{band.split('_')[-1]}_mag')
    aper_magnitudes.add_column((cat[f'{band}_MAG_APER2']),
                                name = f'{band.split('_')[-1]}_mag_2p0_aper')
    magnitude_errs.add_column((cat[f'{band}_MAG_AUTO']),
                          name = f'{band.split('_')[-1]}_magerr')
    SNRs.add_column(cat[f'{band}_FLUX_AUTO']/cat[f'{band}_FLUXERR_AUTO'],
                    name = f'{band.split('_')[-1]}')

colors = Table()
for i, band_hi in enumerate(photometric_bands):
    for band_lo in photometric_bands[i + 1:]:
        color_column = cat[f'{band_hi}_MAG_APER2'] - cat[f'{band_lo}_MAG_APER2']
        color_err_column = np.sqrt(cat[f'{band_hi}_MAGERR_APER2'] ** 2 + cat[f'{band_lo}_MAGERR_APER2'] ** 2)
        colors.add_columns([color_column, color_err_column],
                   names = [f'{band_hi.split('_')[-1]}{band_lo.split('_')[-1]}_col', f'{band_hi.split('_')[-1]}{band_lo.split('_')[-1]}_col_err'])


mstar = 10**cat['lp_mass_med']
mstar_err = 10**cat['lp_mass_med_max68'] - 10**cat['lp_mass_med_min68']

R50 = cat['FLUX_RADIUS']

redshift = cat['lp_zBEST']
redshift_err = cat['lp_zPDF_u68'] - cat['lp_zPDF_l68']

In [20]:
#deredden colors
color_excesses = COSMOSClassicCat['lp_dust']
R_vals = [4.47, 3.63, 2.67, 2.08, 1.54, 1.21, 0.87, 0.56, 0.36]

dereddened_colors = Table()
for i, band_hi in enumerate(photometric_bands):
    for j, band_lo in enumerate(photometric_bands[i + 1:]):
        dereddened_colors.add_column(colors[f'{band_hi.split('_')[-1]}{band_lo.split('_')[-1]}_col'] - (R_vals[i] - R_vals[j]) * color_excesses,
                                     name = f'{band_hi.split('_')[-1]}{band_lo.split('_')[-1]}_col_dered')

In [21]:
#quality_cuts

SNR_thresh = 5
SNR_mask = (SNRs['r'] > SNR_thresh) &\
           (SNRs['i'] > SNR_thresh) &\
           (SNRs['z'] > SNR_thresh) &\
           (SNRs['Y'] > SNR_thresh) &\
           (SNRs['J'] > SNR_thresh) &\
           (SNRs['H'] > SNR_thresh) &\
           (SNRs['Ks'] > SNR_thresh)

redshift_mask = (redshift >= 0.002) & (redshift < 1)

color_lo = -1
color_hi = 3
color_mask = np.ones(len(cat), dtype = bool)
for color in dereddened_colors.colnames:
    if ~color.endswith('_err'):
        color_mask = color_mask &\
                    (dereddened_colors[color] > color_lo) & (dereddened_colors[color] < color_hi) &\
                    ~np.isnan(dereddened_colors[color])
        
log_mstar_lo = 6
log_mstar_hi = 12
log_mstar_mask = (np.log10(mstar) > log_mstar_lo) & (np.log10(mstar) < log_mstar_hi)

R50_mask = R50 < 10

r_mag_mask = magnitudes['r_mag'] > 18

/var/tmp/lsajkov/ipykernel_2722084/3911572392.py:18: DeprecationWarning: Bitwise inversion '~' on bool is deprecated. This returns the bitwise inversion of the underlying int object and is usually not what you expect from negating a bool. Use the 'not' operator for boolean negation or ~int(x) if you really want the bitwise inversion of the underlying int.
  if ~color.endswith('_err'):


In [22]:
total_mask = SNR_mask &\
             redshift_mask &\
             color_mask &\
             log_mstar_mask &\
             R50_mask &\
             r_mag_mask

In [25]:
COSMOS_training_catalog = Table([IDs,
                                 ra, dec,
                                 mstar, redshift,
                                 redshift,
                                 R50],
                          names = ['ID',
                                   'ra', 'dec',
                                   'mstar', 'mstar_err',
                                   'redshift',
                                   'half_light_radius'])

COSMOS_training_catalog.add_columns(colors.columns,
                                      indexes = [3] * len(colors.colnames))

COSMOS_training_catalog.add_columns(dereddened_colors.columns,
                                      indexes = [3] * len(dereddened_colors.colnames))

COSMOS_training_catalog.add_columns(magnitudes.columns,
                                      indexes = [3] * len(magnitudes.colnames))

COSMOS_training_catalog.add_columns(aper_magnitudes.columns,
                                      indexes = [3] * len(aper_magnitudes.colnames))

COSMOS_training_catalog = COSMOS_training_catalog[total_mask]

In [28]:
COSMOS_training_catalog.write('/data2/lsajkov/mpdg/data_products/COSMOS/COSMOS_SOM_panchromatic_catalog_06Aug24.fits',
                                overwrite = True)